<b>Назначение:</b> Классификация пола людей с выбранных изображений

In [31]:
from transformers import AutoModelForImageClassification, AutoImageProcessor
from PIL import Image
import pandas as pd
import torch
from tqdm import tqdm
from collections import Counter

GENDER_MODEL_PATH = "rizvandwiki/gender-classification"
IMAGES_INFO_TABLE = '../data/images_info.csv'

 0 - female
 1 - male

In [27]:
model = AutoModelForImageClassification.from_pretrained(GENDER_MODEL_PATH).to('cuda')
preprocessor = AutoImageProcessor.from_pretrained(GENDER_MODEL_PATH)

In [28]:
images_info = pd.read_csv(IMAGES_INFO_TABLE, sep=';')

In [29]:
label_map = {
    0: 'female',
    1: 'male'
}

tmp_labels = []
for i in tqdm(range(images_info.shape[0])):
    cur_image_path = f"../{images_info['relative_path'][i]}/{images_info['images_name'][i]}"
    cur_image = Image.open(cur_image_path)
    img_tensor = torch.unsqueeze(torch.tensor(preprocessor(cur_image)['pixel_values'][0]),0).to('cuda')

    image_gender_label = torch.argmax(model(img_tensor).logits.detach().cpu()).tolist()

    #print(image_gender_label)

    tmp_labels.append(label_map[image_gender_label])

100%|██████████| 50000/50000 [48:16<00:00, 17.26it/s]  


In [32]:
print(Counter(tmp_labels))

Counter({'female': 28751, 'male': 21249})


In [33]:
images_info['label'] = tmp_labels

In [34]:
images_info.to_csv(IMAGES_INFO_TABLE, sep=';', index=False)